In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jupyros
import rospy
import rospkg
from turtlesim.srv import Spawn
from random import randint
from math import radians
from ipywidgets import Image
from ipycanvas import hold_canvas, Canvas
from time import time

In [3]:
rospy.init_node("superturtle")

# Turtlesim

In [7]:
# !rosrun turtlesim turtlesim_node
# !rosrun turtlesim turtle_teleop_key
# jupyros.client('spawn', Spawn)

In [8]:
def get_random_turtle():
    turtles_svg = ['hydro', 'indigo', 'kinetic', 'lunar']
    turtles_png = ['box-turtle', 'diamondback', 'electric', 'fuerte', 'groovy', 'hydro', 'indigo', 'jade',
                   'kinetic', 'lunar', 'melodic', 'noetic', 'robot-turtle', 'sea-turtle', 'turtle']
    
    # Find path to a random turtle image
    r = rospkg.RosPack()
    turtlesim_path = r.get_path('turtlesim')
    turtle_path = turtlesim_path + '/images/' + turtles_png[randint(0, len(turtles_png)-1)] + '.png'

    turtle_img = Image.from_file(turtle_path)

    return turtle_img


def spawn_turtle():
    spawn_canvas = Canvas(width=1600, height=1200, layout=dict(width="100%"))
    
    # Water
    spawn_canvas.fill_style = "#4556FF"
    spawn_canvas.fill_rect(0, 0, spawn_canvas.width, spawn_canvas.height)
    
    # Turtle
    turtle_img = get_random_turtle()   
    turtle_size = 100  
    turtle_canvas = Canvas(width=turtle_size, height=turtle_size)   
    turtle_canvas.draw_image(turtle_img, width=turtle_size)
    
    # Center of turtle and canvas from top left
    x = (spawn_canvas.width - turtle_size) / 2
    y = (spawn_canvas.height - turtle_size) / 2
    
    # Water canvas
    spawn_canvas.draw_image(turtle_img, x, y, width=turtle_size)

    return turtle_canvas, spawn_canvas

In [9]:
import rostopic

pubs, _ = rostopic.get_topic_list()
pubs

# Retrieve name of last turtle spawned
last_turtle = pubs[-1][0].split('/')[1]
print(last_turtle)

turtle1


### Spawn turtle

In [10]:
turtle_canvas, water_canvas = spawn_turtle()
# display(water_canvas)

### Translate and rotate turtle

In [11]:
def move_turtle(turtle_canvas, water_canvas, x, y, theta, turtle_size=100): 
    with hold_canvas(water_canvas):
        movement = "Moving to {}, {}, with {} radians"
        print(movement.format(x,y,theta))
        
        # Clear old animation step
        water_canvas.clear()

        # New animation drawings
        water_canvas.fill_style = "#4556FF"
        water_canvas.fill_rect(0, 0, water_canvas.width, water_canvas.height)
        
        # Transform canvas
        water_canvas.translate(x, y)
        water_canvas.rotate(-theta)
        
        # Offsets for turtle center
        x_offset = -turtle_size / 2
        y_offset = -turtle_size / 2
        
        water_canvas.draw_image(turtle_canvas, 
                                x=x_offset, 
                                y=y_offset, 
                                width=turtle_size)
        
        # Revert transformation
        water_canvas.rotate(theta)
        water_canvas.translate(-x, -y)
        
    return water_canvas

In [12]:
# from time import sleep

# x = water_canvas.width // 2
# y = water_canvas.height // 2

# for i in range(360):
#     move_turtle(turtle_canvas, water_canvas, x+i, y+i, theta=radians(i))
#     sleep(0.01)

In [13]:
from geometry_msgs.msg import Twist
from turtlesim.msg import Pose

topic_name = '/turtle1/pose'

# jupyros.publish(topic_name, Pose)

In [14]:
# rospy.init_node('listen')

# callback_pose = lambda msg: move_turtle(turtle_canvas, water_canvas, 
#                                         msg.x, 
#                                         msg.y, 
#                                         msg.theta)

# jupyros.subscribe(topic_name, Pose, callback_pose)

In [15]:
def convert_xy_units(x, y):
    convert_factor = 11.08
    x_convert = x / convert_factor * water_canvas.width
    y_convert = (convert_factor - y) / convert_factor * water_canvas.height
    
    return x_convert, y_convert

In [16]:
elapsed_time = 0 # seconds
start_time = time()

def callback_pose(msg):
    global start_time
    
    elapsed_time = time() - start_time

    if elapsed_time > 0.1:
        start_time = time()
        x_canvas, y_canvas = convert_xy_units(msg.x, msg.y)
        move_turtle(turtle_canvas, water_canvas, x_canvas, y_canvas, msg.theta)
        

In [17]:
display(water_canvas)

Canvas(height=1200, layout=Layout(width='100%'), width=1600)

In [18]:
jupyros.subscribe(topic_name, Pose, lambda msg: callback_pose(msg))